##### [ 한글 데이터셋  RNN ] <hr>
- 데이터셋 : Kopora의 NAVER Sentiment Movie Corpus

- [1] 데이터 준비 <hr>

In [2]:
#### ===> 모듈 로딩
#%pip install Korpora

from Korpora import Korpora 
import pandas as pd 
import numpy as np 

In [3]:
#### ===> 데이터 로딩
corpous = Korpora.load('nsmc')
print(corpous)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\gy931\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\gy9

In [4]:
nsmcDF = pd.DataFrame(corpous.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [5]:
nsmIter=nsmcDF.itertuples()
for item in nsmIter:
    print(item, item.text, item.label)
    break

Pandas(Index=0, text='굳 ㅋ', label=1) 굳 ㅋ 1


- [2] 데이터셋 준비 <hr>

In [6]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, corpus):
        nsmcDF = pd.DataFrame(corpus).fillna('')

        x_data = nsmcDF['text'].values
        self.x_data = x_data
        self.y_data = nsmcDF['label'].values

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return y, x

In [10]:
# class CustomDataset(Dataset):
#     def __init__(self, texts, labels, vocab, tokenizer):
#         super().__init__()
#         self.texts = texts
#         self.labels = labels
#         self.vocab = vocab
#         self.tokenizer = tokenizer
        
        
#     def __len__(self):
#         return len(self.labels)
        
#     def __getitem__(self, idx):
#         text = self.texts.iloc[idx]
#         label = self.labels.iloc[idx]
#         return self.vocab(self.tokenizer(text)), label

In [7]:
trainDS = CustomDataset(corpous.train)
testDS = CustomDataset(corpous.test)

In [8]:
print(f'trainDS =>  {len(trainDS)}개   testDS =>  {len(testDS)}개')

trainDS =>  150000개   testDS =>  50000개


In [9]:
for label, text in trainDS:
    print(label, text)
    break

for label, text in testDS:
    print(label, text)
    break

0 아 더빙.. 진짜 짜증나네요 목소리
1 굳 ㅋ


- [2] 단어 사전 생성 <hr>
    * 토큰화 진행 ==> 형태소 분석기 선택 
    * 단어 사전 

- [2-1] 토큰화 관련 준비

In [16]:
### ===> 모듈 로딩
from konlpy.tag import Okt
import torchtext
from torchtext.vocab import build_vocab_from_iterator
torchtext.disable_torchtext_deprecation_warning()

In [14]:
### ===> 토큰관련 특별 문자
UNK = '<UNK>'
PAD = '<PAD>'

In [13]:
### 토큰화 인스턴스 생성
tokenizer = Okt()

In [17]:
### ===> 토큰 제너레이터 함수 : 데이터 추출하여 토큰화 
def yield_tokens(data_iter):
    for label, text in data_iter:
        # 라벨, 텍스트 --> 텍스트 토큰화
        yield tokenizer.morphs(text, stem=True)

- [2-2] 토큰화 ===> 단어/어휘 사전 생성

In [18]:
### ===> 토큰화 및 단어/어휘 사전 생성
VOCAB = build_vocab_from_iterator(
    yield_tokens(trainDS),
    min_freq=2,
    specials= [PAD, UNK],
    special_first=True
)

### <UNK> 인덱스 설정
VOCAB.set_default_index(VOCAB[UNK])

In [19]:
### VOCAB 메서드 
VOCAB.get_itos()[:9], VOCAB.get_stoi()['영화'], VOCAB.get_stoi()[UNK]

(['<PAD>', '<UNK>', '.', '이', '영화', '보다', '하다', '의', '..'], 4, 1)

In [20]:
### ===> 텍스트 >>>> 정수 인코딩
text_pipeline = lambda x: VOCAB(tokenizer.morphs(text, stem=True))

### ===> 레이틀 >>> 정수 인코딩 (0~3)
label_pipeline = lambda x: int(x) - 1

- [2-3] 인코딩 & 디코딩 인덱싱 

In [25]:
### 인코딩 : 문자 >>>> 숫자로 변환
token_to_id ={ label : id  for label, id in VOCAB.get_stoi().items()}

### 디코딩 : 숫자 >>>> 문자로 변환
id_to_token ={ id : label  for label, id in VOCAB.get_stoi().items()}

- [3] 데이터 로더 생성 <hr>

In [22]:
### ===> 모듈로딩
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pad_sequence
import torch  


In [23]:
### ===> 실행 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def collate_batch(batch,vocab):
    label_list, target_list= [], []
    for label,target in batch:
         label_list.append(vocab[label])
         processed_text = torch.tensor(target, dtype=torch.int64)
         target_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.float32)
    target_list = pad_sequence(target_list, batch_first=True, padding_value=0)
    return label_list.to(DEVICE), target_list.to(DEVICE)



In [ ]:
### ===> 학습용, 검증용, 테스트용 DataSet 준비 
BATCH_SIZE = 64

### 학습용, 검증용, 테스트용 Dataset, DataLoader 준비
num_train = int(len(trainDS) * 0.95)
print(f' num_train :{num_train}')

split_trainDS, split_validDS= random_split( trainDS, [num_train, len(trainDS) - num_train])
print(f' len(split_trainDS) :{len(split_trainDS)}')
print(f' len(split_validDS) :{len(split_validDS)}')

trainDL = DataLoader( split_trainDS, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch ,generator=)
validDL = DataLoader( split_validDS, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch )
testDL  = DataLoader( testDS,        batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch )



 num_train :142500
 len(split_trainDS) :142500
 len(split_validDS) :7500


In [26]:
print(f' len(trainDL) :{len(trainDL)*BATCH_SIZE}')
print(f' len(validDL) :{len(validDL)*BATCH_SIZE}')
print(f' len(testDL) :{len(testDL)* BATCH_SIZE}')

 len(trainDL) :142528
 len(validDL) :7552
 len(testDL) :50048
